Connect to [RACE](http://www.gitan.ulg.ac.be/cms/index.php?page=donnees-de-stareso#1.) database (*Rapid Assessment of the marine Coastal Environment*) and perform a simple data query.     
We will use the [psycopg2](https://wiki.postgresql.org/wiki/Psycopg2) package.

In [1]:
import psycopg2
import logging
import getpass
import sys
import matplotlib.pyplot as plt

# Prerequisites

1. You need to have a user name and password to access the database.
2. If you're not using the ULg network (wired or wireless), it is necessary to use the ULg Virtual Private Network (VPN), otherwise the connection won't be authorised.

# Configuration

## Logging

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

## Define the query

If you're not familiar with the querly language, use [`pgadmin`](https://www.pgadmin.org/) (graphical administration tool for [PostgreSQL](https://www.postgresql.org/)) to prepare the request using the graphical interface.<br>
Most of the interesting data are stored in one of these tables:
* data,
* timeseries,
* profiledata.

In the following query, we want to obtain the *wind speed* variable acquired from April 1st 2017 on.

`data.dasa_codeds` and<br> 
`timeseries.codeds`<br>
are the *Primary Keys* (PK) that allow us to join the 2 tables `data` and `timeseries`. 

In [3]:
windquery = """SELECT
  data.datavalue,
  data.daytime,
  data.dataparam_codeparam
FROM
  public.data,
  public.timeseries
WHERE
  data.dasa_codeds = timeseries.codeds AND
  timeseries.tssta_codesta = 'INSJD' AND
  data.daytime > '2017-04-01 00:00:00' AND
  data.dataparam_codeparam = 'windspeed'
GROUP BY
  data.dataparam_codeparam,
  data.datavalue,
  data.daytime
ORDER BY
  data.daytime;
  """

We just need to make sure no *new lines* are present in the string:

In [4]:
windquery = windquery.replace('\n', ' ')

## Connection setup

You need to have an access to the database.    
Here you will have to write your credentials, but another solution would be to read them from a local file on your machine.

In [6]:
dbhost = getpass.getpass(prompt="Enter host:")

Enter host:········


In [7]:
dbuser = getpass.getpass(prompt="Enter user name:")

Enter user name:········


In [8]:
dbpa = getpass.getpass(prompt="Enter word:")

Enter word:········


In [9]:
conn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpa, dbname="race_db_v2")

OperationalError: could not connect to server: Connection timed out
	Is the server running on host "10.16.7.18" and accepting
	TCP/IP connections on port 5432?


If you get an `OperationalError`, check if your VPN connection was properly set up.

### Connection status

In [ ]:
if conn.status == psycopg2.extensions.STATUS_READY:
    logging.info("Status: ready")
elif conn.status == psycopg2.extensions.STATUS_BEGIN:
    logging.info("Status: begin")
elif conn.status == psycopg2.extensions.STATUS_IN_TRANSACTION:
    logging.info("Status: in transaction")
elif conn.status == psycopg2.extensions.STATUS_PREPARED:
    logging.info("Status: prepared")
else:
    logging.error("Uncorrect status")

# Execute request

Open a [cursor](https://en.wikipedia.org/wiki/Cursor_(databases) to perform database operations:

In [ ]:
cur = conn.cursor()

Execute a command: this creates a new table

In [ ]:
cur.execute(windquery)

## Read the data into a variable

In [ ]:
data = cur.fetchall()

Close the cursor and the connection:

In [ ]:
cur.close()
conn.close()

In [ ]:
data

## Parse the data

The data obtained are in the form of a list of tuples:

In [ ]:
logging.debug(type(data))
logging.debug(data[-1])

We convert the list of tuples to two separate lists:
* one for the wind speed mesurements,
* the other for the datatime objects.

In [ ]:
windspeed, datetimelist = [], []
for dd in data:
    windspeed.append(dd[0])
    datetimelist.append(dd[1])

# Make a plot

The `plot` function accepts the datetime as an argument.

In [ ]:
fig = plt.figure(figsize=(12, 10))
plt.plot(datetimelist, windspeed)
plt.xlabel("Time")
plt.ylabel("Wind speed\n(m/s)", rotation=0, ha='right')
fig.autofmt_xdate()
plt.show()
plt.close()

In [ ]:
import numpy as np
import pandas as pd
import pandas_highcharts
from pandas_highcharts.display import display_charts

Create a pandas data frame to use with highcharts.   
We have to set the index properly for further visualisation.

In [ ]:
windspeedDF = pd.DataFrame(
    {'datetime': np.array(datetimelist),
     'windspeed': windspeed
    })
windspeedDF = windspeedDF.set_index('datetime')

In [ ]:
display_charts(windspeedDF, chart_type="stock", title="Wind speed")